In [5]:
import torch
from LSTM import LSTM
import Dataloader

model = LSTM(hidden_dim=75)

#loss_fn = torch.nn.MSELoss(size_average=False)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())#, lr=learning_rate)

num_epochs = 5  


N = 50
N_minibatch_loss = 0.0

dataloader = Dataloader.Dataloader()

inputs,targets = dataloader.load_data('../pa4Data/train.txt')

val_inputs,val_targets = dataloader.load_data('../pa4Data/val.txt')

# Track the loss across training
total_loss = []
avg_minibatch_loss = []
val_losses = []
epochs = []
minibatches = []
val_count=0
save_state_dict=None

for epoch in range(num_epochs): 
    model.clear_hidden() # zero out hidden/memory state
    
    for i in range(len(inputs)):
        model.train()
        
        # Zero out gradient
        optimizer.zero_grad()
        
        outputs = model(inputs[i])
        
        loss = criterion(outputs, targets[i].long())
          
        # Backward pass
        loss.backward()
        
        # Update parameters
        optimizer.step()

        # Add this iteration's loss to the total_loss
        total_loss.append(loss.item())
        N_minibatch_loss += loss.item()
        
        if i % N == 0: 
            # Print the loss averaged over the last N mini-batches    
            N_minibatch_loss /= N
            print('Epoch %d, average minibatch %d loss: %.3f' %
                (epoch + 1, i, N_minibatch_loss))
            
            # Add the averaged loss over N minibatches and reset the counter
            avg_minibatch_loss.append(N_minibatch_loss)
            N_minibatch_loss = 0.0
            
            # validation
            print('...validating...')
            val_loss=0.0
            with torch.no_grad():
                for v in range(len(val_inputs)):
                    val_outputs = model(val_inputs[v])
                    val_loss += criterion(val_outputs, val_targets[v]).item()
            val_loss/=len(val_inputs)
            val_losses.append(val_loss)
            
            epochs.append(epoch+1)
            minibatches.append(i)
            
            print('Epoch %d, average validation loss: %.3f' %
                (epoch + 1, val_loss))
            
            # early stopping
            if val_loss>val_losses[-1]:
                if val_count==0:
                    save_state_dict = model.state_dict()
                val_count+=1
                if val_count==5:
                    break  
            
    print("Finished", epoch + 1, "epochs of training")

if save_state_dict:
    PATH = "./output/{}/best.pt".format(model_name)
    torch.save(save_state_dict, PATH)

    


Epoch 1, average minibatch 0 loss: 0.090
...validating...
Epoch 1, average validation loss: 4.523
Epoch 1, average minibatch 50 loss: 4.042
...validating...
Epoch 1, average validation loss: 3.802
Epoch 1, average minibatch 100 loss: 3.167
...validating...
Epoch 1, average validation loss: 4.150
Epoch 1, average minibatch 150 loss: 3.656
...validating...
Epoch 1, average validation loss: 3.786
Epoch 1, average minibatch 200 loss: 3.538
...validating...
Epoch 1, average validation loss: 3.811
Epoch 1, average minibatch 250 loss: 3.338
...validating...
Epoch 1, average validation loss: 3.774
Epoch 1, average minibatch 300 loss: 3.395
...validating...
Epoch 1, average validation loss: 3.786
Epoch 1, average minibatch 350 loss: 3.611
...validating...
Epoch 1, average validation loss: 3.778
Epoch 1, average minibatch 400 loss: 3.334
...validating...
Epoch 1, average validation loss: 3.726
Epoch 1, average minibatch 450 loss: 3.420
...validating...
Epoch 1, average validation loss: 3.641
Epo

Epoch 2, average validation loss: 2.510
Epoch 2, average minibatch 300 loss: 2.216
...validating...
Epoch 2, average validation loss: 2.484
Epoch 2, average minibatch 350 loss: 2.107
...validating...
Epoch 2, average validation loss: 2.600
Epoch 2, average minibatch 400 loss: 2.071
...validating...
Epoch 2, average validation loss: 2.466
Epoch 2, average minibatch 450 loss: 1.778
...validating...
Epoch 2, average validation loss: 2.547
Epoch 2, average minibatch 500 loss: 1.519
...validating...
Epoch 2, average validation loss: 2.590
Epoch 2, average minibatch 550 loss: 1.892
...validating...
Epoch 2, average validation loss: 2.484
Epoch 2, average minibatch 600 loss: 2.314
...validating...
Epoch 2, average validation loss: 2.498
Epoch 2, average minibatch 650 loss: 1.928
...validating...
Epoch 2, average validation loss: 2.698
Epoch 2, average minibatch 700 loss: 1.747
...validating...
Epoch 2, average validation loss: 2.631
Epoch 2, average minibatch 750 loss: 1.601
...validating...


Epoch 3, average minibatch 550 loss: 1.620
...validating...
Epoch 3, average validation loss: 2.338
Epoch 3, average minibatch 600 loss: 2.050
...validating...
Epoch 3, average validation loss: 2.362
Epoch 3, average minibatch 650 loss: 1.654
...validating...
Epoch 3, average validation loss: 2.437
Epoch 3, average minibatch 700 loss: 1.509
...validating...
Epoch 3, average validation loss: 2.418
Epoch 3, average minibatch 750 loss: 1.395
...validating...
Epoch 3, average validation loss: 2.425
Epoch 3, average minibatch 800 loss: 1.572
...validating...
Epoch 3, average validation loss: 2.464
Epoch 3, average minibatch 850 loss: 1.333
...validating...
Epoch 3, average validation loss: 2.449
Epoch 3, average minibatch 900 loss: 1.585
...validating...
Epoch 3, average validation loss: 2.449
Epoch 3, average minibatch 950 loss: 1.400
...validating...
Epoch 3, average validation loss: 2.502
Epoch 3, average minibatch 1000 loss: 1.649
...validating...
Epoch 3, average validation loss: 2.408

Epoch 4, average validation loss: 2.348
Epoch 4, average minibatch 850 loss: 1.174
...validating...
Epoch 4, average validation loss: 2.293
Epoch 4, average minibatch 900 loss: 1.464
...validating...
Epoch 4, average validation loss: 2.292
Epoch 4, average minibatch 950 loss: 1.312
...validating...
Epoch 4, average validation loss: 2.329
Epoch 4, average minibatch 1000 loss: 1.549
...validating...
Epoch 4, average validation loss: 2.312
Epoch 4, average minibatch 1050 loss: 1.424
...validating...
Epoch 4, average validation loss: 2.295
Epoch 4, average minibatch 1100 loss: 1.622
...validating...
Epoch 4, average validation loss: 2.261
Epoch 4, average minibatch 1150 loss: 1.781
...validating...
Epoch 4, average validation loss: 2.324
Epoch 4, average minibatch 1200 loss: 1.544
...validating...
Epoch 4, average validation loss: 2.289
Epoch 4, average minibatch 1250 loss: 1.331
...validating...
Epoch 4, average validation loss: 2.308
Epoch 4, average minibatch 1300 loss: 1.471
...validat

Epoch 5, average minibatch 1100 loss: 1.531
...validating...
Epoch 5, average validation loss: 2.200
Epoch 5, average minibatch 1150 loss: 1.682
...validating...
Epoch 5, average validation loss: 2.269
Epoch 5, average minibatch 1200 loss: 1.479
...validating...
Epoch 5, average validation loss: 2.233
Epoch 5, average minibatch 1250 loss: 1.277
...validating...
Epoch 5, average validation loss: 2.252
Epoch 5, average minibatch 1300 loss: 1.402
...validating...
Epoch 5, average validation loss: 2.195
Epoch 5, average minibatch 1350 loss: 1.633
...validating...
Epoch 5, average validation loss: 2.199
Epoch 5, average minibatch 1400 loss: 1.498
...validating...
Epoch 5, average validation loss: 2.197
Epoch 5, average minibatch 1450 loss: 1.407
...validating...
Epoch 5, average validation loss: 2.215
Epoch 5, average minibatch 1500 loss: 1.398
...validating...
Epoch 5, average validation loss: 2.186
Epoch 5, average minibatch 1550 loss: 1.416
...validating...
Epoch 5, average validation lo

In [6]:
import pandas as pd

#train_loss_df = {'epoch':epochs,'minibatch':minibatches,'loss':avg_minibatch_loss}
#val_loss_df = {'epoch':epochs,'minibatch':minibatches,'loss':val_losses}

train_loss_df = pd.DataFrame({'loss':avg_minibatch_loss})
val_loss_df = pd.DataFrame({'loss':val_losses})

model_name="lstm" + str(model.hidden_dim)
val_loss_file = "./output/{}/validation_loss.csv".format(model_name)
train_loss_file = "./output/{}/train_loss.csv".format(model_name)
scores_file = "./output/{}/scores.csv".format(model_name)

train_loss_df.to_csv(train_loss_file)
val_loss_df.to_csv(val_loss_file)

In [7]:
PATH = "./output/{}/lstm.pt".format(model_name)
torch.save(model.state_dict(), PATH)